In [17]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.stats.multitest as smm

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
raw = pd.read_csv("h:\\MyProj\\Courcera\\Ml-mipt\\course4\\week4Genome\\gene_high_throughput_sequencing.csv", sep=",") 
raw.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [26]:
raw['LOC643837'].mean()

2.4639219315000003

In [4]:
early = raw.loc[raw['Diagnosis'] == 'early neoplasia']
normal = raw.loc[raw['Diagnosis'] == 'normal']
cancer = raw.loc[raw['Diagnosis'] == 'cancer']
raw.Diagnosis.value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

##### Часть 1: применение t-критерия Стьюдента
В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

для групп normal (control) и early neoplasia (treatment)
для групп early neoplasia (control) и cancer (treatment)
В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.

In [14]:
normal_early = 0
for column in normal:
    if (column != 'Patient_id') & (column != 'Diagnosis'):
        statistic, pvalue = stats.ttest_ind(normal[column], early[column], equal_var = False)
        if (pvalue<0.05):
            normal_early = normal_early + 1
            
print("normal_early:", normal_early)

normal_early: 1575


In [16]:
early_cancer = 0
for column in early:
    if (column != 'Patient_id') & (column != 'Diagnosis'):
        statistic, pvalue = stats.ttest_ind(early[column], cancer[column], equal_var = False)
        if (pvalue<0.05):
            early_cancer = early_cancer + 1
            
print("early_cancer:", early_cancer)

early_cancer: 3490


##### Часть 2: поправка методом Холма
В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5

Обратите внимание, что

применять поправку на множественную проверку нужно ко всем значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия.
при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)!

In [46]:
def get_pvalues(aa, bb, alpha = 0.025, method = 'holm'):
    data = []
    for column in aa:
        if (column != 'Patient_id') & (column != 'Diagnosis'):
            statistic, pvalue = stats.ttest_ind(aa[column], bb[column], equal_var = False)
            amean = aa[column].mean()
            bmean = bb[column].mean()
            if (amean > bmean):
                fc = amean/bmean
            else:
                fc = bmean/amean
            data.append([column, fc, pvalue])
#            data.append([column, pvalue])
    data_frame = pd.DataFrame.from_records(data)
    data_frame.columns = ['gene_name', 'fold_change_abs', 'pvalue']
#    data_frame.columns = ['gene_name', 'pvalue']
    reject, p_corrected, a1, a2 = smm.multipletests(data_frame.pvalue, alpha, method)
    data_frame['pvalue_corrected'] = p_corrected
    data_frame['reject'] = reject
    return data_frame

In [47]:
raw_normal_early = get_pvalues(normal,early,  0.025)
df_normal_early = raw_normal_early.loc[raw_normal_early['reject'] == True]
df_normal_early.head()

,gene_name,fold_change_abs,pvalue,pvalue_corrected,reject
7244,PCSK4,1.509785,7.955435e-07,0.012527,True
9820,EEF1A2,1.974868,8.498742e-08,0.001338,True


In [49]:
#raw_normal_early.sort_values(by='pvalue_corrected', ascending=True)
df_normal_early.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 7244 to 9820
Data columns (total 5 columns):
gene_name           2 non-null object
fold_change_abs     2 non-null float64
pvalue              2 non-null float64
pvalue_corrected    2 non-null float64
reject              2 non-null bool
dtypes: bool(1), float64(3), object(1)
memory usage: 82.0+ bytes


In [53]:
raw_early_cancer = get_pvalues(early,cancer,  0.025)
raw_early_cancer.head()

,gene_name,fold_change_abs,pvalue,pvalue_corrected,reject
0,LOC643837,1.148593,0.413735,1.0,False
1,LOC100130417,1.064871,0.653429,1.0,False
2,SAMD11,1.278517,0.079556,1.0,False
3,NOC2L,1.092635,0.287581,1.0,False
4,KLHL17,1.036574,0.463292,1.0,False


In [55]:
df_early_cancer = raw_early_cancer.loc[(raw_early_cancer['reject'] == True) & (raw_early_cancer['fold_change_abs']> 1.5)]
df_early_cancer.head()

,gene_name,fold_change_abs,pvalue,pvalue_corrected,reject
47,GABRD,1.843860,2.276043e-07,0.003575,True
283,EXTL1,2.383210,1.087046e-07,0.001709,True
316,CD164L2,1.997381,1.002085e-06,0.015708,True
1105,NES,1.693428,1.041679e-07,0.001637,True
1238,FMO2,1.795640,1.103311e-06,0.017292,True


In [56]:
df_early_cancer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77 entries, 47 to 15703
Data columns (total 5 columns):
gene_name           77 non-null object
fold_change_abs     77 non-null float64
pvalue              77 non-null float64
pvalue_corrected    77 non-null float64
reject              77 non-null bool
dtypes: bool(1), float64(3), object(1)
memory usage: 3.1+ KB


##### Часть 3: поправка методом Бенджамини-Хохберга
Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от ${H_0}$, когда они есть, и будут чаще отклонять ${H_0}$, когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5.

In [57]:
raw_normal_early_bh = get_pvalues(normal,early,  0.025, 'fdr_bh')
df_normal_early_bh = raw_normal_early_bh.loc[raw_normal_early_bh['reject'] == True]
df_normal_early_bh.head()

,gene_name,fold_change_abs,pvalue,pvalue_corrected,reject
4627,TMEM63C,1.663826,2.222228e-06,0.008749,True
7244,PCSK4,1.509785,7.955435e-07,0.006264,True
8250,CACNG8,1.754951,2.015472e-06,0.008749,True
9820,EEF1A2,1.974868,8.498742e-08,0.001338,True


In [58]:
df_normal_early_bh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 4627 to 9820
Data columns (total 5 columns):
gene_name           4 non-null object
fold_change_abs     4 non-null float64
pvalue              4 non-null float64
pvalue_corrected    4 non-null float64
reject              4 non-null bool
dtypes: bool(1), float64(3), object(1)
memory usage: 164.0+ bytes


In [59]:
raw_early_cancer_bh = get_pvalues(early,cancer,  0.025, 'fdr_bh')
raw_early_cancer_bh.head()

,gene_name,fold_change_abs,pvalue,pvalue_corrected,reject
0,LOC643837,1.148593,0.413735,0.675195,False
1,LOC100130417,1.064871,0.653429,0.836406,False
2,SAMD11,1.278517,0.079556,0.288873,False
3,NOC2L,1.092635,0.287581,0.563007,False
4,KLHL17,1.036574,0.463292,0.712214,False


In [60]:
df_early_cancer_bh = raw_early_cancer_bh.loc[(raw_early_cancer_bh['reject'] == True) & (raw_early_cancer_bh['fold_change_abs']> 1.5)]
df_early_cancer_bh.head()

,gene_name,fold_change_abs,pvalue,pvalue_corrected,reject
8,ISG15,2.458867,7.402400e-04,0.017556,True
47,GABRD,1.843860,2.276043e-07,0.000086,True
75,AJAP1,1.567814,7.993294e-04,0.018270,True
104,CA6,1.736762,7.806213e-04,0.018078,True
127,MASP2,1.750200,9.522413e-05,0.005049,True


In [61]:
df_early_cancer_bh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 524 entries, 8 to 15703
Data columns (total 5 columns):
gene_name           524 non-null object
fold_change_abs     524 non-null float64
pvalue              524 non-null float64
pvalue_corrected    524 non-null float64
reject              524 non-null bool
dtypes: bool(1), float64(3), object(1)
memory usage: 21.0+ KB
